<a href="https://colab.research.google.com/github/shahisha/End-to-End-Machine-Learning/blob/main/Bagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.utils import resample

In [ ]:
df=pd.read_csv('/content/churndata_processed.csv')
df

,months,multiple,gb_mon,security,backup,protection,support,unlimited,contract,paperless,...,payment_Credit Card,payment_Mailed Check,internet_type_DSL,internet_type_Fiber Optic,internet_type_None,offer_Offer A,offer_Offer B,offer_Offer C,offer_Offer D,offer_Offer E
0,0.00,0,0.094118,0,0,1,0,0,0.0,1,...,0,0,1,0,0,0,0,0,0,0
1,0.00,1,0.200000,0,1,0,0,1,0.0,1,...,1,0,0,1,0,0,0,0,0,1
2,0.25,1,0.611765,0,0,0,0,1,0.0,1,...,0,0,0,1,0,0,0,0,1,0
3,0.25,0,0.141176,0,1,1,0,1,0.0,1,...,0,0,0,1,0,0,0,1,0,0
4,0.50,1,0.164706,0,0,0,0,1,0.0,1,...,0,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,1.00,0,0.000000,0,0,0,0,0,1.0,1,...,0,0,0,0,1,0,0,0,0,0
7039,0.25,1,0.282353,1,0,1,1,1,0.5,1,...,0,1,0,0,0,0,0,1,0,0
7040,1.00,1,0.694118,0,1,1,0,1,0.5,1,...,1,0,0,1,0,0,0,0,0,0
7041,0.00,0,0.200000,1,0,0,0,1,0.0,1,...,0,0,1,0,0,0,0,0,0,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   months                     7043 non-null   float64
 1   multiple                   7043 non-null   int64  
 2   gb_mon                     7043 non-null   float64
 3   security                   7043 non-null   int64  
 4   backup                     7043 non-null   int64  
 5   protection                 7043 non-null   int64  
 6   support                    7043 non-null   int64  
 7   unlimited                  7043 non-null   int64  
 8   contract                   7043 non-null   float64
 9   paperless                  7043 non-null   int64  
 10  monthly                    7043 non-null   float64
 11  satisfaction               7043 non-null   float64
 12  churn_value                7043 non-null   int64  
 13  payment_Credit Card        7043 non-null   int64

In [ ]:
df.describe()

,months,multiple,gb_mon,security,backup,protection,support,unlimited,contract,paperless,...,payment_Credit Card,payment_Mailed Check,internet_type_DSL,internet_type_Fiber Optic,internet_type_None,offer_Offer A,offer_Offer B,offer_Offer C,offer_Offer D,offer_Offer E
count,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,...,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000
mean,0.433551,0.421837,0.241358,0.286668,0.344881,0.343888,0.290217,0.673719,0.377396,0.592219,...,0.390317,0.054664,0.234559,0.430924,0.216669,0.073832,0.116996,0.058924,0.085475,0.114298
std,0.398231,0.493888,0.240223,0.452237,0.475363,0.475038,0.453895,0.468885,0.424234,0.491457,...,0.487856,0.227340,0.423753,0.495241,0.412004,0.261516,0.321438,0.235499,0.279607,0.318195
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.035294,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.250000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.750000,1.000000,0.317647,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
df[df.duplicated()]

,months,multiple,gb_mon,security,backup,protection,support,unlimited,contract,paperless,...,payment_Credit Card,payment_Mailed Check,internet_type_DSL,internet_type_Fiber Optic,internet_type_None,offer_Offer A,offer_Offer B,offer_Offer C,offer_Offer D,offer_Offer E
1945,0.00,0,0.0,0,0,0,0,0,0.0,0,...,1,0,0,0,1,0,0,0,0,0
2480,0.00,0,0.0,0,0,0,0,0,0.0,1,...,1,0,0,0,1,0,0,0,0,1
2728,0.00,0,0.0,0,0,0,0,0,0.0,0,...,1,0,0,0,1,0,0,0,0,0
3335,0.00,0,0.0,0,0,0,0,0,0.0,0,...,0,0,0,0,1,0,0,0,0,1
3498,0.75,0,0.0,0,0,0,0,0,1.0,0,...,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6841,1.00,1,0.0,0,0,0,0,0,1.0,0,...,1,0,0,0,1,0,0,0,0,0
6855,0.00,0,0.0,0,0,0,0,0,0.0,0,...,1,0,0,0,1,0,0,0,0,0
6877,0.00,0,0.0,0,0,0,0,0,0.0,0,...,1,0,0,0,1,0,0,0,0,0
6879,1.00,0,0.0,0,0,0,0,0,1.0,1,...,1,0,0,0,1,0,0,0,0,0


In [ ]:
df.drop_duplicates(inplace = True)

In [ ]:
df.duplicated().sum()

0

In [ ]:
df['churn_value'].value_counts()

0    5084
1    1867
Name: churn_value, dtype: int64

In [ ]:
x = df.drop(['churn_value'],axis=1)
y = df['churn_value']

In [ ]:
df0=df[df['churn_value']==0]
df1=df[df['churn_value']==1]

In [ ]:
sample=resample(df1,n_samples=5084)

In [ ]:
balanced=pd.concat([sample,df0])

In [ ]:
balanced['churn_value'].value_counts()

1    5084
0    5084
Name: churn_value, dtype: int64

Splitting

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=42)

In [ ]:
classf = RandomForestClassifier()
classf.fit(x_train, y_train)

RandomForestClassifier()

In [ ]:
y_pred = classf.predict(x_test)

In [ ]:
print(metrics.confusion_matrix(y_test, y_pred))

[[1217   21]
 [  79  421]]


Bagging Classifier

In [ ]:
bagging = BaggingClassifier(base_estimator = RandomForestClassifier(), random_state = 42)
bagging.fit(x_train, y_train)

BaggingClassifier(base_estimator=RandomForestClassifier(), random_state=42)

In [ ]:
y_pred_bag = bagging.predict(x_test)

In [ ]:
result = model_selection.cross_val_score(classf, x, y)
print(result)

[0.95111431 0.94388489 0.94820144 0.95323741 0.94244604]


In [ ]:
max(result)

0.9532374100719424

In [ ]:
y_pred_new = bagging.predict(x_train)

In [ ]:
print(metrics.classification_report(y_train, y_pred_new))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      3846
           1       0.99      0.95      0.97      1367

    accuracy                           0.99      5213
   macro avg       0.99      0.98      0.98      5213
weighted avg       0.99      0.99      0.99      5213



F1 Score is the harmonic mean of precision and recall. It takes both false positive and false negatives into account. Therefore, it performs well on an imbalanced dataset.

F1 score gives the same weightage to recall and precision.